In [72]:
import pandas as pd
import numpy as np
import math

# Cargar todas las hojas en un diccionario de DataFrames
excel_tablas = "Tablas PER 2000 y 2020.xlsx" 
hojas = pd.read_excel(excel_tablas, sheet_name=None, header=1)  # Cargar todas las pestañas
xlsx = pd.ExcelFile(excel_tablas) # Cargar el archivo Excel y sirve para obtener los nombres de las pestañas

# Obtener los nombres de todas las pestañas
nombres_pestañas = xlsx.sheet_names

nombre_tabla = input("¿Qué Tabla de Mortalidad quieres usar? Elige entre:\n"
    "'PERM2000C', 'PERF2000C', 'PERM2000P', 'PERF2000P',\n"
    "'PERM_2020_Indiv_2Orden', 'PERM_2020_Indiv_1Orden',\n"
    "'PERF_2020_Indiv_2Orden', 'PERF_2020_Indiv_1Orden',\n"
    "'PERM_2020_Colectivos_2Orden', 'PERM_2020_Colectivos_1Orden',\n"
    "'PERF_2020_Colectivos_2Orden', 'PERF_2020_Colectivos_1Orden': ")

g = int(input("Introduce el año de nacimiento del asegurado/a: "))


In [74]:
def leer_tablas(nombre_tabla):
    """
    La función leer_tablas toma el nombre de una pestaña del archivo Excel y devuelve un DataFrame
    que contiene los datos de esa Tabla de Mortalidad. La primera fila se utiliza como encabezado y 
    la primera columna se establece como índice (edad x+t).
    """
    # Leer la tabla de la hoja especificada
    df = hojas[nombre_tabla]
    df.set_index(df.columns[0], inplace=True)  # Establecer la primera columna como índice
    return df

def generacion(g, nombre_tabla):

         
    if nombre_tabla in ['PERM2000C',
                            'PERF2000C',
                            'PERM2000P',
                            'PERF2000P',]:
        x_mas_t = 2000 - g

        if g > 2000:
            raise ValueError("La generación no puede ser mayor a 2000.")

        
    
    elif nombre_tabla in ['PERM_2020_Indiv_2Orden',
                            'PERM_2020_Indiv_1Orden',
                            'PERF_2020_Indiv_2Orden',
                            'PERF_2020_Indiv_1Orden',
                            'PERM_2020_Colectivos_2Orden',
                            'PERM_2020_Colectivos_1Orden',
                            'PERF_2020_Colectivos_2Orden',
                            'PERF_2020_Colectivos_1Orden']:
        x_mas_t = 2012 - g

        if g > 2012:
            raise ValueError("La generación no puede ser mayor a 2012.")
 
    return x_mas_t



def q_x(qx, mejora, t):
    '''
    La función q_x(x, mejora) recibe la probabilidad de muerte qx de la tabla de mortalidad y el factor de mejora de ésta
    Devuelve la probabilidad de muerte qx ajustada por el factor de mejora.
    '''
    return qx*math.exp(-mejora*t)

In [75]:
edad_inicio = generacion(g, nombre_tabla)
edad_inicio

50

In [ ]:
x_mas_t_list = list(range(edad_inicio, len(hojas[nombre_tabla]))) #Lista de edades x+t
edad_t = list(range(0, len(x_mas_t_list))) #Lista de edades t

# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame(columns=['Edad', 'x+t'])
resultados['x+t'] = x_mas_t_list
resultados['Edad'] = edad_t  # Asignar la lista de edades al DataFrame


qx_ajustado = []

for i in range(edad_inicio, len(hojas[nombre_tabla])):
    qx_ajustado.append(q_x(hojas[nombre_tabla].iloc[i, 1], #Coge de la tabla original las qx+t tabla base
                            hojas[nombre_tabla].iloc[i, 2], #Coge de la tabla original las mejoras
                            i - edad_inicio)) # i - edad_inicio es el tiempo t

# Asignar la lista de qx ajustados al DataFrame
resultados.insert(2, 'qx+t ajustado', qx_ajustado)

resultados.set_index('Edad', inplace=True)  # Establecer la columna 'Edad' como índice

In [78]:
resultados['lx'] = 0  
resultados.loc[0, 'lx'] = 1000000  

# Calculamos lx y dx fila por fila
for i in range(1, len(resultados)):
    resultados.loc[i, 'lx'] = resultados.loc[i-1, 'lx'] * (1 - resultados.loc[i-1, 'qx+t ajustado'])

resultados['dx'] = resultados['lx'] * resultados['qx+t ajustado']
resultados

C:\Users\pablo\AppData\Local\Temp\ipykernel_1456\1082342199.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '993112.5555411397' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc[i, 'lx'] = resultados.loc[i-1, 'lx'] * (1 - resultados.loc[i-1, 'qx+t ajustado'])


,x+t,qx+t ajustado,lx,dx
Edad,,,,
0,50,0.003281,1000000.000000,3281.000000
1,51,0.003618,996719.000000,3606.444459
2,52,0.003883,993112.555541,3856.010337
3,53,0.004563,989256.545204,4513.953855
4,54,0.004766,984742.591350,4693.549371
...,...,...,...,...
61,111,0.589211,184.564478,108.747421
62,112,0.617937,75.817057,46.850165
63,113,0.648064,28.966892,18.772400
